In [9]:
import os
import platform
import time
from dotenv import load_dotenv
import smtplib
from email.mime.text import MIMEText
import sqlite3

load_dotenv()

# Ping function
def ping(host):
    param = "-n 1" if platform.system().lower() == "windows" else "-c 1"
    command = f"ping {param} {host}"
    response = os.system(command)
    return response == 0

# Alert function
def send_email(subject, body):
    sender = "busdata123@gmail.com"
    receiver = "bbal1535111@gmail.com"
    password = os.getenv("EMAIL_PASSWORD")
    msg = MIMEText(body)
    msg["Subject"] = subject
    msg["From"] = sender
    msg["To"] = receiver
    with smtplib.SMTP("smtp.gmail.com", 587) as server:
        server.ehlo()  # Can be omitted
        server.starttls()
        server.ehlo()  # Can be omitted
        server.login(sender, password)
        server.sendmail(sender, receiver, msg.as_string())

# Logging function
def log_status(host, status):
    conn = sqlite3.connect("network_monitor.db")
    cursor = conn.cursor()
    cursor.execute("CREATE TABLE IF NOT EXISTS server_status (host TEXT, status TEXT, timestamp TEXT)")
    cursor.execute("INSERT INTO server_status (host, status, timestamp) VALUES (?, ?, ?)",
                   (host, status, time.strftime("%Y-%m-%d %H:%M:%S")))
    conn.commit()
    conn.close()


# Main monitoring loop
def monitor_servers(servers):
    # while True:
        for server in servers:
            if ping(server):
                print(f"{server} is up!")
                log_status(server, "up")
                send_email(f"Server Up: {server}", f"The server {server} is currently up.")
            else:    
                print(f"{server} is down!")
                log_status(server, "down")
                send_email(f"Server Down: {server}", f"The server {server} is currently down.")
        # time.sleep(300)  # Check every 5 minutes

# Example usage
if __name__ == "__main__":
    servers = ["google.com", "github.com", "example.com"]
    monitor_servers(servers)

google.com is up!
github.com is up!
example.com is up!


In [6]:
import os
from dotenv import load_dotenv
from email.mime.text import MIMEText
import smtplib

# Load environment variables from .env file
load_dotenv()

# Alert function
def send_email(subject, body):
    sender = "busdata123@gmail.com"
    receiver = "bbal1535111@gmail.com"
    password = os.getenv("EMAIL_PASSWORD")
    msg = MIMEText(body)
    msg["Subject"] = subject
    msg["From"] = sender
    msg["To"] = receiver
    with smtplib.SMTP("smtp.gmail.com", 587) as server:
        server.ehlo()  # Can be omitted
        server.starttls()
        server.ehlo()  # Can be omitted
        server.login(sender, password)
        server.sendmail(sender, receiver, msg.as_string())

# Test the function
send_email("Test Email", "This is a test email.")

In [4]:
import sqlite3
import time

# Logging function
def log_status(host, status):
    conn = sqlite3.connect("network_monitor.db")
    cursor = conn.cursor()
    cursor.execute("CREATE TABLE IF NOT EXISTS server_status (host TEXT, status TEXT, timestamp TEXT)")
    cursor.execute("INSERT INTO server_status (host, status, timestamp) VALUES (?, ?, ?)",
                   (host, status, time.strftime("%Y-%m-%d %H:%M:%S")))
    conn.commit()
    conn.close()

# Example usage
log_status("example.com....", "up")

In [1]:
pip install Flask

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [8]:
from flask import Flask, render_template
import sqlite3

app = Flask(__name__)

@app.route("/")
def dashboard():
    conn = sqlite3.connect("network_monitor.db")
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM server_status ORDER BY timestamp DESC")
    data = cursor.fetchall()
    conn.close()
    return render_template("dashboard.html", data=data)

if __name__ == "__main__":
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1